# データの準備

## データのロードと分割

In [1]:
from keras.datasets import mnist

(X_trainval, y_trainval), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=0)

## データの前処理

In [3]:
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_train = X_train.astype("float32") / 255

X_val = X_val.reshape((X_val.shape[0], 28, 28, 1))
X_val = X_val.astype("float32") / 255

X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_test = X_test.astype("float32") / 255

In [4]:
X_train.shape, X_val.shape, X_test.shape

((45000, 28, 28, 1), (15000, 28, 28, 1), (10000, 28, 28, 1))

In [5]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [6]:
y_train.shape, y_val.shape, y_test.shape

((45000, 10), (15000, 10), (10000, 10))

## ネットワークの構築

In [11]:
from keras import backend, layers, models

backend.clear_session()

# ネットワークの初期化
model = models.Sequential()

model.add(layers.Conv2D(filters=32, # 畳み込み演算によって計算するフィルタの数
                        kernel_size=(3, 3),
                        padding="valid", # パディングなし
                        activation="relu",
                        input_shape=(28, # image_height
                                     28, # image_width
                                     1)  # channel
                       )) 

model.add(layers.MaxPool2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=64,
                        kernel_size=(3, 3),
                        activation="relu"))
                                                                                
model.add(layers.MaxPool2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=64,
                        kernel_size=(3, 3),
                        activation="relu"))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

# 学習

In [12]:
#モデルをコンパイル
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [13]:
model.fit(X_train,
          y_train,
          epochs=5,
          batch_size=64,
          validation_data=(X_val, y_val))

Train on 45000 samples, validate on 15000 samples
Epoch 1/5
45000/45000 [==============================] - 20s 440us/step - loss: 0.2230 - acc: 0.9309 - val_loss: 0.1136 - val_acc: 0.9653
Epoch 2/5
45000/45000 [==============================] - 19s 425us/step - loss: 0.0590 - acc: 0.9823 - val_loss: 0.0545 - val_acc: 0.9815
Epoch 3/5
45000/45000 [==============================] - 26s 575us/step - loss: 0.0405 - acc: 0.9880 - val_loss: 0.0444 - val_acc: 0.9862
Epoch 4/5
45000/45000 [==============================] - 23s 511us/step - loss: 0.0295 - acc: 0.9910 - val_loss: 0.0400 - val_acc: 0.9891
Epoch 5/5
45000/45000 [==============================] - 19s 433us/step - loss: 0.0223 - acc: 0.9930 - val_loss: 0.0355 - val_acc: 0.9897


# プーリング層を使わない場合どうなるか？

In [ ]:
backend.clear_session()

model_no_max_pool = models.Sequential()

model_no_max_pool.add(layers.Conv2D(32, (3, 3), input_shape=(28,28,1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), input_shape=(28,28,1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), input_shape=(28,28,1)))

model_no_max_pool.add(layers.Flatten())
model_no_max_pool.add(layers.Dense(64, activation="relu"))
model_no_max_pool.add(layers.Dense(10, activation="softmax"))

model_no_max_pool.summary()